In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [38]:
import os

llm = ChatOpenAI(
    model = "DMXAPI-DeepSeek-V3-Fast",
    api_key = os.getenv("DMX_API_KEY"),
    base_url = os.getenv("DMX_BASE_URL")
)

In [39]:
prompt = PromptTemplate.from_template("10乘以{num}等于多少？")

In [40]:
chain = prompt | llm

In [41]:
chain.invoke({"num" : 10})


AIMessage(content='10乘以10等于100。  \n\n计算过程如下：  \n\\[ 10 \\times 10 = 100 \\]  \n\n如果有其他问题，欢迎继续提问！ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 9, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'DeepSeek-V3-Fast', 'system_fingerprint': None, 'id': 'chatcmpl-7b270fe93a8a44449747a5cf1656ea90', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a870f86c-5eeb-4382-bfd9-5331a72e8918-0', usage_metadata={'input_tokens': 9, 'output_tokens': 36, 'total_tokens': 45, 'input_token_details': {}, 'output_token_details': {}})

In [42]:
chain.invoke(8) # 如果模板中只有一个变量，也可以直接传递该变量的值

AIMessage(content='要计算 \\( 10 \\times 8 \\)，可以按照以下步骤进行：\n\n1. **理解乘法**：乘法是重复加法的简便方法。\\( 10 \\times 8 \\) 表示将 10 加 8 次，或者将 8 加 10 次。\n\n2. **计算**：\n   \\[\n   10 \\times 8 = 80\n   \\]\n\n3. **验证**：\n   - 将 10 加 8 次：\\( 10 + 10 + 10 + 10 + 10 + 10 + 10 + 10 = 80 \\)\n   - 将 8 加 10 次：\\( 8 + 8 + 8 + 8 + 8 + 8 + 8 + 8 + 8 + 8 = 80 \\)\n\n因此，最终答案为：\n\n\\[\n\\boxed{80}\n\\]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 197, 'prompt_tokens': 9, 'total_tokens': 206, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'DeepSeek-V3-Fast', 'system_fingerprint': None, 'id': 'chatcmpl-e1fa0ce6152944519d1ee97a1a820a68', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--9cf117bb-5eed-49ef-956d-be292de09763-0', usage_metadata={'input_tokens': 9, 'output_tokens': 197, 'total_tokens': 206, 'input_token_details': {}, 'output_token_details': {}})

RunnablePassthrough 是一个 runnable 类型的对象，具有以下特征：

1. 基本操作

- 执行一个简单的传递函数，将输入值直接传递给输出

- 可以独立使用 invoke() 方法执行

2. 用例

- "在需要通过链式步骤传递数据且无需修改时非常方便"

- 可与其他组件结合，构建复杂的数据管道

- 特别有助于在添加新字段时保持原始输入的完整性

3. 输入管理

- 接受字典类型输入

- 也可以处理单个值
 
- 在整个链条中保持数据结构的完整性

In [43]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num" : 100})

{'num': 100}

In [44]:
# 使用 RunnablePassthrough 创建链
runnable_chain = {"num" :RunnablePassthrough()} | prompt | llm

In [45]:
runnable_chain.invoke(5)

AIMessage(content='10乘以5等于50。  \n\n计算过程：  \n10 × 5 = 50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 9, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'DeepSeek-V3-Fast', 'system_fingerprint': None, 'id': 'chatcmpl-d1e396c958824a1c9f011404cf530112', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--81ea3748-9c5e-4b4a-8cee-e1198c38bf4e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 19, 'total_tokens': 28, 'input_token_details': {}, 'output_token_details': {}})

RunnablePassthrough.assign()

将输入中的键/值对与分配的新键/值对进行合并。

1. 输入数据: {"num": 1}

2. lambda函数处理:

函数接收字典 x = {"num": 1}

计算 x["num"] * 3 = 1 * 3 = 3

3. 字段添加:

使用 assign() 方法将新字段 "new_num": 3 添加到原始数据中

4. 最终结果:

{'num': 1, 'new_num': 3}

In [46]:
RunnablePassthrough.assign(new_num = lambda x : x["num"] * 3).invoke({"num" : 1})

{'num': 1, 'new_num': 3}


使用RunnableParallel高效并行执行

" RunnableParallel 是一个用于同时执行多个 Runnable 对象的工具，旨在简化需要并行处理的流程。它将输入数据分配到不同组件中，收集各组件的结果，并将这些结果整合为统一输出。这一功能使其成为优化那些任务可独立且并行执行的工作流程的强大工具。"

1. 并行处理

同时执行多个 Runnable 对象，减少并行化任务所需的时间。

2. 统一输出管理系统

将所有并行执行的结果整合成一个统一的输出，从而简化后续处理流程。

3. 灵活

可处理多种输入类型，高效分配工作负载，从而支持复杂工作流程。

In [47]:
from langchain_core.runnables import RunnableParallel

runnableparallel = RunnableParallel(
    default = RunnablePassthrough(),
    extend = RunnablePassthrough().assign(mult = lambda x : x["num"] * 3),
    modified = lambda x : x["num"] + 1
)

runnableparallel.invoke({"num" : 1})

{'default': {'num': 1}, 'extend': {'num': 1, 'mult': 3}, 'modified': 2}

链条同样适用于RunnableParallel

In [48]:
chain_1 = {"country" : RunnablePassthrough()} | PromptTemplate.from_template("{country}首都是哪里？") | llm

chain_2 = {"country" : RunnablePassthrough()} | PromptTemplate.from_template("{country}面积多大？") | llm

parallel_combined = RunnableParallel(capital = chain_1,area = chain_2)

parallel_combined.invoke({"country" : "中国"})

{'capital': AIMessage(content='中国的首都是**北京**。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'DeepSeek-V3-Fast', 'system_fingerprint': None, 'id': 'chatcmpl-23a3daa61c924c64a3074b91cb7d1e14', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--251840c9-d889-44c7-83b2-ee7bba479dba-0', usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {}, 'output_token_details': {}}),
 'area': AIMessage(content='中国的国土面积约为 **960万平方公里**，是世界第三大国家（仅次于俄罗斯和加拿大）。这一数据包括陆地面积和内陆水域（如湖泊、河流），但不包括领海和专属经济区。  \n\n若包含领海和争议区域（如台湾地区、南海诸岛等），中国的总面积可达到约 **1,040万平方公里**（不同统计口径可能略有差异）。  \n\n需要说明的是，台湾地区是中国领土不可分割的一部分，中国政府对其拥有主权。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 12, 'total_tokens': 112, 'completion_tokens_deta

动态处理 RunnableLambda 函数

" RunnableLambda 是一个灵活的工具，能让开发人员利用 lambda 函数定义自定义数据转换逻辑。它通过实现快速简便的自定义处理工作流程，简化了定制数据管道的创建，同时确保了最小的设置开销。"

1. 可定制数据转换功能

允许用户通过 lambda 函数自定义转换输入数据的逻辑，具有极高的灵活性。

2. 轻巧且简便

提供了一种简便的方法来实现临时处理，无需定义大量的类或函数。

In [49]:
from datetime import datetime
# 必须定义一个参数，即使这个参数用不到，这是langchain框架的要求
def get_today(a):
    return datetime.today().strftime("%b-%d")



In [50]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

prompt_lambda = PromptTemplate.from_template("给我{n}个在{today}出生的历史名人，要写出出生日期。")

chain_lambda = ({"today" : RunnableLambda(get_today),"n" : RunnablePassthrough()}) | prompt_lambda | llm | StrOutputParser()

chain_lambda.invoke({"n" : 3})

"以下是3位于8月2日出生的历史名人及其出生日期：  \n\n1. **皮埃尔·德·顾拜旦（Pierre de Coubertin）**  \n   - 出生日期：1863年8月2日  \n   - 现代奥林匹克运动会创始人，被誉为“奥林匹克之父”。  \n\n2. **詹姆斯·鲍德温（James Baldwin）**  \n   - 出生日期：1924年8月2日  \n   - 美国著名作家、社会活动家，代表作《向苍天呼吁》。  \n\n3. **彼得·奥图尔（Peter O'Toole）**  \n   - 出生日期：1932年8月2日  \n   - 爱尔兰裔英国演员，因主演《阿拉伯的劳伦斯》闻名。  \n\n如果需要更多信息，请告诉我！"

通过 itemgetter 提取特定键

" itemgetter 是 Python 的 operator 模块提供的实用函数，具有以下特点和优势："

1. 核心功能

- 能高效地从字典、元组和列表中提取特定键或索引的值

- 可以同时提取多个键或索引

- 支持函数式编程风格

2. 性能优化

- 对于重复的键访问操作，这种索引方式比常规索引更高效

- 优化内存使用

- 处理大型数据集所带来的性能优势

3. 在 LangChain 里的应用

- 链式组合中的数据筛选
- 从复杂输入结构中进行选择性提取
- 与其他可运行对象结合进行数据预处理

In [51]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

def length_function(text):
    return len(text)


def _multiple_length_function(text1,text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"],_dict["text2"])


prompt = ChatPromptTemplate.from_template("{a}+{b}是多少？")

model = ChatOpenAI(model = "glm-4.5",base_url = os.getenv("DMX_BASE_URL"),api_key = os.getenv("DMX_API_KEY"))

chain = ({
    "a" : itemgetter("world1") | RunnableLambda(length_function),
    "b" : {"text1" : itemgetter("world1"),"text2" : itemgetter("world2")}
    |
    RunnableLambda(multiple_length_function),
}
| prompt | model)

In [52]:
chain.invoke({"world1" : "hello","world2" : "world"})

AIMessage(content='<think>\n 首先，用户的问题是“5+25是多少？”这是一个简单的加法问题。我需要计算5加25的结果。\n> \n> 回想一下基本的加法运算：5加25等于30。因为5 + 25 = 30。\n> \n> 为了确保准确性，我可以分解计算：25加5。25加5是30，因为25的个位数是5，加上5后变成10，所以进位1，十位数的2加1是3，因此是30。\n> \n> 或者，更简单地说，5 + 25 = 30。这是一个基础数学问题，不需要复杂计算。\n> \n> 现在，考虑用户的意图。用户可能是在测试我的基本数学能力，或者是一个孩子在练习加法。无论哪种情况，我应该给出直接、准确的答案。\n> \n> 我的回应应该清晰、简洁。既然问题很简单，我可以直接回答“5+25等于30。”\n> \n> 为了更友好，我可以添加一些鼓励性的语言，比如“很简单，5加25等于30。”但不要过度复杂化。\n> \n> 最后，确认没有误解：问题是用中文写的，“5+25是多少？”意思是“5+25是多少？”所以我应该用中文回应以保持一致。\n> \n> 因此，我的回应是：“5+25等于30。”</think>\n\n5+25等于30。这是一个简单的加法运算，5加25的结果是30。如果您有其他数学问题，欢迎随时问！ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 464, 'prompt_tokens': 14, 'total_tokens': 478, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'text_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': 0, 'image_tokens': 0}, 'input_tokens': 0, 'o